In [9]:
import sys
print(f"Python Version {sys.version.split()[0]}")


# define linear alg problem
#!{sys.executable} -m pip install --upgrade pip
#!{sys.executable} -m pip install scipy
from scipy import linalg
import numpy as np
import numpy.linalg as la

Python Version 3.12.9


In [14]:
def jacobi(A, b, x, N_max = 150):
    """Jacobi iteration"""

    if A.shape[0] != A.shape[1]:
        raise RuntimeError("Error: Matrix must be square!")

    N = A.shape[0]

    # check for diagonal dominance
    
    if any(np.sum(abs(A), axis=1) - 2 * abs(np.diag(A)) > 0):
        print("Warning: Matrix is not diagonally dominant!")

    # For Jacobi iteration, the iteration matrix is G = I -D^{-1}*A
    
    G = A.copy()
    for i in range(N):
        G[i, :] /= G[i, i]
    G = np.eye(N) - G

    print(f"rho(G) = {max(abs(la.eigvals(G)))}")

    r = b - np.matmul(A, x)
    rho = la.norm(r)

    TOL = 1.0e-09
    n_iter = 0
    while rho > TOL and n_iter <= N_max:
        x += r / np.diag(A)
        #x = G @ x + b / np.diag(A)
        r = b - np.matmul(A, x)
        rho = la.norm(r)
        n_iter += 1

        print(
            f" k = {n_iter}, |r| = {rho:e}"
        )


In [32]:
def gauss_seidel(A: np.ndarray, b: np.ndarray, x: np.ndarray, conv_tol: float = 1.e-09, N_max: int = 1000) -> np.ndarray:
    
    N = A.shape[0]

    for k in range(1,N_max):

        dx = 0
        for i in range(N):
            xik = x[i]
            el = b[i]
            for j in range(N):
                el -= A[i,j] * x[j]
            x[i] += el / A[i,i]
            dx += (x[i] - xik) * (x[i] - xik)
            
        dx /= N
        dx = np.sqrt(dx)
        
        if (dx < conv_tol):
            r_final = la.norm(b - A @ x)
            print(f"Gauss Seidel iterations to reach tolerance 10^-9: {k}\n Final residual norm: {r_final:e}")
            return x, k, dx, r_final
    raise RuntimeError(f"Gauss-Seidel: Maximum Iterations ({N_max}) reached")
    

## Question 1:

In [35]:
# Use the existing jacobi implementation to solve (H + I)x = b with x0 = 0
from scipy import linalg
import numpy as np
import numpy.linalg as la

N = 10
H = linalg.hilbert(N)
A = H + np.eye(N)
x_true = np.ones(N)
b = H @ x_true
x0 = np.zeros(N)

# Compute spectral radius of Jacobi iteration matrix
D = np.diag(A)
G = np.eye(N) - (A / D[:, None])
rho_G = max(abs(la.eigvals(G)))
print(f"Spectral radius of Jacobi iteration matrix: {rho_G:.4f}")

# Run Jacobi iteration
jacobi(A, b, x0, N_max=1000)

Spectral radius of Jacobi iteration matrix: 0.9663
rho(G) = 0.9662538182093741
 k = 1, |r| = 4.379422e+00
 k = 2, |r| = 4.251441e+00
 k = 3, |r| = 4.102125e+00
 k = 4, |r| = 3.965352e+00
 k = 5, |r| = 3.831039e+00
 k = 6, |r| = 3.701906e+00
 k = 7, |r| = 3.576935e+00
 k = 8, |r| = 3.456241e+00
 k = 9, |r| = 3.339602e+00
 k = 10, |r| = 3.226905e+00
 k = 11, |r| = 3.118009e+00
 k = 12, |r| = 3.012788e+00
 k = 13, |r| = 2.911118e+00
 k = 14, |r| = 2.812879e+00
 k = 15, |r| = 2.717955e+00
 k = 16, |r| = 2.626234e+00
 k = 17, |r| = 2.537609e+00
 k = 18, |r| = 2.451974e+00
 k = 19, |r| = 2.369229e+00
 k = 20, |r| = 2.289277e+00
 k = 21, |r| = 2.212023e+00
 k = 22, |r| = 2.137375e+00
 k = 23, |r| = 2.065247e+00
 k = 24, |r| = 1.995553e+00
 k = 25, |r| = 1.928210e+00
 k = 26, |r| = 1.863141e+00
 k = 27, |r| = 1.800267e+00
 k = 28, |r| = 1.739515e+00
 k = 29, |r| = 1.680813e+00
 k = 30, |r| = 1.624092e+00
 k = 31, |r| = 1.569285e+00
 k = 32, |r| = 1.516327e+00
 k = 33, |r| = 1.465157e+00
 k = 3

## Question 2: 

In [36]:
N = 10
H = linalg.hilbert(N)
A = H + np.eye(N)
x_true = np.ones(N)
b = H @ x_true
x0 = np.zeros(N)

try:
    x_sol, k_final, dx_final, r_final = gauss_seidel(A, b, x0)
except RuntimeError as e:
    print(f"Runtime Error: {e}")



Gauss Seidel iterations to reach tolerance 10^-9: 14
 Final residual norm: 7.455768e-10


In [22]:
# Use the existing gauss_seidel implementation to solve (H + I)x = b with x0 = 0, and report convergence info
from scipy import linalg
import numpy as np
import numpy.linalg as la

N = 10
H = linalg.hilbert(N)
A = H + np.eye(N)
x_true = np.ones(N)
b = H @ x_true
x0 = np.zeros(N)

class GSConvergence(Exception):
    pass

def gauss_seidel_convergence(A, b, x, conv_tol=1e-9, N_max=1000):
    N = A.shape[0]
    for k in range(1, N_max+1):
        x_old = x.copy()
        dx = 0
        for i in range(N):
            xik = x[i]
            el = b[i]
            for j in range(N):
                el -= A[i,j] * x[j]
            x[i] += el / A[i,i]
            dx += (x[i] - xik) ** 2
        dx /= N
        dx = np.sqrt(dx)
        if dx < conv_tol:
            r_final = la.norm(b - A @ x)
            print(f"Gauss-Seidel iterations to reach ||xk+1 - xk|| < 1e-9: {k}")
            print(f"Final ||xk+1 - xk||: {dx:.2e}")
            print(f"Final residual norm ||rk_final||: {r_final:.2e}")
            return x, k, dx, r_final
    raise GSConvergence(f"Gauss-Seidel: Maximum Iterations ({N_max}) reached")

x_sol, k_final, dx_final, r_final = gauss_seidel_convergence(A, b, x0)

Gauss-Seidel iterations to reach ||xk+1 - xk|| < 1e-9: 14
Final ||xk+1 - xk||: 6.18e-10
Final residual norm ||rk_final||: 7.46e-10


## Question 3:

In [37]:
# implementation of minimum residual method

N = 10
H = linalg.hilbert(N)
A = H + np.eye(N)
x_true = np.ones(N)
b = H @ x_true
x0 = np.zeros(N)

def min_residual(A, b, x0, tol=1e-9, N_max=1000):
    x = x0.copy()
    for k in range(1, N_max+1):
        r = b - A @ x
        Ar = A @ r
        alpha = np.dot(Ar, r) / np.dot(Ar, Ar) if np.dot(Ar, Ar) != 0 else 0
        x_new = x + alpha * r
        r_new = b - A @ x_new
        norm_r = la.norm(r_new)
        if norm_r < tol:
            print(f"Minimum Residual iterations to reach ||rk|| < 1e-9: {k}")
            print(f"Final residual norm: {norm_r:.2e}")
            return x_new, k, norm_r
        x = x_new
    raise RuntimeError(f"Minimum Residual: Maximum Iterations ({N_max}) reached")

x_minres, k_minres, r_minres = min_residual(A, b, x0)

Minimum Residual iterations to reach ||rk|| < 1e-9: 19
Final residual norm: 7.92e-10
